# Modeling Sanity Check: Why is my model not converging?

In this notebook, we'll test our neural network on a simple binary classification dataset to validate that our DataLoader is working correctly.

In [1]:
import dask.dataframe as dd
import pandas as pd 
import torch
import linecache 
import csv
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Let's define our custom data class and make sure everything is being streamed in correctly

In [2]:
df = pd.read_csv('datatest.csv')
labels = pd.read_csv('labelstest.csv')

In [3]:
class CustomDataset(Dataset):
    def __init__(self, filename, labelname):
        self._filename = filename
        self._labelname = labelname
        self._total_data = 0
        
        with open(filename, "r") as f:
            self._total_data = len(f.readlines()) - 1
    
    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 2)
        label = linecache.getline(self._labelname, idx + 2)
        
        csv_data = csv.reader([line])
        csv_label = csv.reader([label])
        
        data = [x for x in csv_data][0]
        label = [x for x in csv_label][0]
        
        return torch.from_numpy(np.array([float(x) for x in data])).float(), [int(float(x)) for x in label][0]
    
    def __len__(self):
        return self._total_data

class CustomDataset2(Dataset):
    def __init__(self, data, labels):
        self.data = pd.read_csv(data)
        self.labels = pd.read_csv(labels)
        
    def __getitem__(self, i):
        return torch.from_numpy(self.data.iloc[i, :].values).float(), self.labels.iloc[i, :].values[0]
    
    def __len__(self):
        return self.data.shape[0]

In [4]:
t = CustomDataset('datatest.csv', 'labelstest.csv')
t2 = CustomDataset2('datatest.csv', 'labelstest.csv')
traindata = DataLoader(t, batch_size=8, num_workers=0)
traindata2 = DataLoader(t, batch_size=8, num_workers=0)

In [5]:
# for X, y in t:
#     print(X, y)

In [6]:
# all([X.item() for X, y in t] == [X.item() for X, y in t2])

Now that we have our dataset, we'll train a simple feedforward neural network and see if it converges. In fact, it should overfit quite a lot 

In [7]:
import torch

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.l1 = nn.Linear(in_features=3, out_features=16),
        self.l2 = nn.Linear(in_features=16, out_features=200),
        self.l3 = nn.Linear(in_features=32, out_features=3),
        
    def forward(self, x):
        x = self.l1(x)
        x = nn.ReLU(x)
        x = self.l2(x)
        x = nn.reLU(x)
        x = self.l3(x)
        x = torch.sigmoid(x)
        return x
    
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=3, bias=True)
  )
)

And now we can train our network

In [9]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

# model = NN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

trainer = create_supervised_trainer(model, optimizer, criterion)

val_metrics = {
    "accuracy": Accuracy(),
    "nll": Loss(criterion)
}

evaluator = create_supervised_evaluator(model, metrics=val_metrics)

log_interval = 3

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(traindata)
    metrics = evaluator.state.metrics
    print(f"Training Results - Epoch: {trainer.state.epoch}  Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['nll']:.2f}")

In [10]:
trainer.run(traindata, max_epochs=100)

Training Results - Epoch: 1  Avg accuracy: 0.78 Avg loss: 0.65
Training Results - Epoch: 2  Avg accuracy: 0.88 Avg loss: 0.51
Training Results - Epoch: 3  Avg accuracy: 0.86 Avg loss: 0.61
Training Results - Epoch: 4  Avg accuracy: 0.89 Avg loss: 0.49
Training Results - Epoch: 5  Avg accuracy: 0.86 Avg loss: 0.42
Training Results - Epoch: 6  Avg accuracy: 0.90 Avg loss: 0.34
Training Results - Epoch: 7  Avg accuracy: 0.89 Avg loss: 0.51
Training Results - Epoch: 8  Avg accuracy: 0.91 Avg loss: 0.40
Training Results - Epoch: 9  Avg accuracy: 0.88 Avg loss: 0.44
Training Results - Epoch: 10  Avg accuracy: 0.90 Avg loss: 0.38
Training Results - Epoch: 11  Avg accuracy: 0.91 Avg loss: 0.41
Training Results - Epoch: 12  Avg accuracy: 0.91 Avg loss: 0.33
Training Results - Epoch: 13  Avg accuracy: 0.91 Avg loss: 0.32
Training Results - Epoch: 14  Avg accuracy: 0.91 Avg loss: 0.31
Training Results - Epoch: 15  Avg accuracy: 0.89 Avg loss: 0.31
Training Results - Epoch: 16  Avg accuracy: 0.91 

State:
	iteration: 11200
	epoch: 100
	epoch_length: 112
	max_epochs: 100
	output: 0.3560422956943512
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [13]:
trainer.run(traindata2, max_epochs=100)

Training Results - Epoch: 1  Avg accuracy: 0.91 Avg loss: 0.24
Training Results - Epoch: 2  Avg accuracy: 0.91 Avg loss: 0.24
Training Results - Epoch: 3  Avg accuracy: 0.89 Avg loss: 0.26
Training Results - Epoch: 4  Avg accuracy: 0.87 Avg loss: 0.29
Training Results - Epoch: 5  Avg accuracy: 0.92 Avg loss: 0.23
Training Results - Epoch: 6  Avg accuracy: 0.91 Avg loss: 0.25
Training Results - Epoch: 7  Avg accuracy: 0.91 Avg loss: 0.24
Training Results - Epoch: 8  Avg accuracy: 0.87 Avg loss: 0.29
Training Results - Epoch: 9  Avg accuracy: 0.91 Avg loss: 0.23
Training Results - Epoch: 10  Avg accuracy: 0.91 Avg loss: 0.23
Training Results - Epoch: 11  Avg accuracy: 0.92 Avg loss: 0.24
Training Results - Epoch: 12  Avg accuracy: 0.91 Avg loss: 0.24
Training Results - Epoch: 13  Avg accuracy: 0.90 Avg loss: 0.26
Training Results - Epoch: 14  Avg accuracy: 0.91 Avg loss: 0.24
Training Results - Epoch: 15  Avg accuracy: 0.87 Avg loss: 0.30
Training Results - Epoch: 16  Avg accuracy: 0.92 

State:
	iteration: 11200
	epoch: 100
	epoch_length: 112
	max_epochs: 100
	output: 0.49707961082458496
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>